imports

In [ ]:
%matplotlib inline
import statsmodels as sm
from statsmodels.formula.api import ols
import numpy as np
import pandas as pd
import pylab as plt
import seaborn as sns
import scipy.stats as stats
sns.set(style="white")

## Getting and exploring the data

In [ ]:
immer_ds = sm.datasets.get_rdataset("immer", "MASS")

In [ ]:
print sleepstudy_ds.__doc__

In [ ]:
immer_data = immer_ds.data
immer_ds.data

In [ ]:
immer_data.describe()

## Two sample t test - are two distributions different?

Is manchuria yielding more crop than trebi?

In [ ]:
manchuria_yield = immer_data.Y1[immer_data.Var == "M"]
trebi_yield = immer_data.Y1[immer_data.Var == "T"]

In [ ]:
sns.distplot(manchuria_yield, rug=True, hist=False, label="manchuria")
sns.distplot(trebi_yield, rug=True, hist=False, label="trebi")

In [ ]:
t, p = stats.ttest_ind(manchuria_yield, trebi_yield)
print t, p

How does it work?

$$t = \frac{\bar {X}_1 - \bar{X}_2}{s_{X_1X_2} \cdot \sqrt{\frac{1}{n_1}+\frac{1}{n_2}}}$$

where $s_{X_1X_2}$ the the common variance

$$ s_{X_1X_2} = \sqrt{\frac{(n_1-1)s_{X_1}^2+(n_2-1)s_{X_2}^2}{n_1+n_2-2}}$$

What does it really mean? Further away the means are the higher the statistic gets

In [ ]:
for rv2_mean in range(0,5,1):
    rvs1 = stats.norm.rvs(loc=0,scale=1,size=500)
    rvs2 = stats.norm.rvs(loc=rv2_mean,scale=1,size=500)
    stats.ttest_ind(rvs1,rvs2)
    f = plt.figure()
    sns.distplot(rvs1, hist=False, label="variable 1")
    sns.distplot(rvs2, hist=False, label="variable 2", axlabel="t = %g, p=%g"%stats.ttest_ind(rvs1,rvs2))
    plt.xlim([-9,9])
    plt.ylim([0, 0.45])

In general the less there is an overlap between the two distributions the higher the statistic

In [ ]:
for rvs_spread in range(1,6,1):
    rvs1 = stats.norm.rvs(loc=0,scale=rvs_spread,size=500)
    rvs2 = stats.norm.rvs(loc=1,scale=rvs_spread,size=500)
    stats.ttest_ind(rvs1,rvs2)
    plt.figure()
    sns.distplot(rvs1, hist=False, label="variable 1")
    sns.distplot(rvs2, hist=False, label="variable 2", axlabel="t = %g, p=%g"%stats.ttest_ind(rvs1,rvs2))
    plt.xlim([-12,12])
    plt.ylim([0, 0.45])

The higher the sample size the higher the statistic.

In [ ]:
for sample_size in range(100,500,100):
    rvs1 = stats.norm.rvs(loc=0,scale=1,size=sample_size)
    rvs2 = stats.norm.rvs(loc=1,scale=1,size=sample_size)
    stats.ttest_ind(rvs1,rvs2)
    plt.figure()
    sns.distplot(rvs1, hist=False, label="variable 1")
    sns.distplot(rvs2, hist=False, label="variable 2", axlabel="t = %g, p=%g"%stats.ttest_ind(rvs1,rvs2))
    plt.xlim([-12,12])
    plt.ylim([0, 0.45])

Sample size not only increases the statistic, but also increases degrees of freedom which are used for calculating the p value.

In [ ]:
dfs = range(10,300, 10)
ps = [stats.t.pdf(2, df) for df in dfs]
plt.plot(ps, dfs)
plt.xlabel("p-value")
plt.ylabel("degrees of freedom")

Even tiniest effect size can be statistically significant if you collect enough samples.

In [ ]:
sample_sizes = range(10, 1000, 10)
ps = [stats.ttest_ind(stats.norm.rvs(loc=0,scale=1,size=sample_size),
                      stats.norm.rvs(loc=0.3,scale=1,size=sample_size))[1] for sample_size in sample_sizes]
plt.semilogx(ps, sample_sizes)
plt.xlabel("p-value")
plt.ylabel("sample size")

## Paired T test

In [ ]:
year1_yield = immer_data.Y1
year2_yield = immer_data.Y2

In [ ]:
sns.distplot(manchuria_yield, rug=True, hist=False, label="1931")
sns.distplot(trebi_yield, rug=True, hist=False, label="1932")
plt.xlabel("crop")

In [ ]:
sns.violinplot(inner=None, data=immer_data)
for y1, y2 in zip(year1_yield, year2_yield):
    plt.plot([0,1], [y1,y2], color="b")

In [ ]:
y_diff = year1_yield-year2_yield
sns.distplot(y_diff,  rug=True, hist=False)

Let's calculate a one sample t-test checking if the distribution of differences is significantly different from zero.

In [ ]:
stats.ttest_1samp(y_diff, 0)

This is equivalent to so called paired two sample t test

In [ ]:
stats.ttest_rel(year1_yield,year2_yield)

What would happen if we ignored the pairing and treat the samples as independent?

In [ ]:
stats.ttest_ind(year1_yield,year2_yield)

The result is similar, but would it always be the case? Let's generate a lot of data and compare the difference between two sample independent and paired tests.

In [ ]:
t_diffs = []
rvs = []
for i in range(1000):
    rvs1 = stats.norm.rvs(loc=0,scale=1,size=15)
    rvs2 = stats.norm.rvs(loc=0.2,scale=1,size=15)
    t_rel, _ = stats.ttest_rel(rvs1,rvs2)
    t_ind, _ = stats.ttest_ind(rvs1,rvs2)
    t_diff = t_rel - t_ind
    t_diffs.append(t_diff)
    rvs.append((rvs1, rvs2))

In [ ]:
sns.distplot(t_diffs)

Note that the distribution is left skewed. This means that independent samples t test has more often higher values than the paired version.

Let's pick and extreme

In [ ]:
rv1, rv2 = rvs[np.array(t_diffs).argmax()]

In [ ]:
sns.violinplot(pd.DataFrame(np.array([rv1, rv2]).T))
for y1, y2 in zip(rv1, rv2):
    plt.plot([0,1], [y1,y2], color="b")

In [ ]:
stats.ttest_rel(rv1, rv2)

In [ ]:
stats.ttest_ind(rv1, rv2)

Using the wrong test can result in a test being less or more significant than it should. However the sign of the stattistic can never change.

## Correlation between values

Are the yields of barley crops in 1931 correlated with yields in 1932

In [ ]:
plt.scatter(x=year1_yield, y=year2_yield)

We can try to fit a line to this plot. A line is defined by the following equation:
$$ y = ax + b $$
where $a$ the slope and $b$ is the intercept of the line.

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(year1_yield, year2_yield)

In [ ]:
plt.scatter(x=year1_yield, y=year2_yield)
plt.plot(np.arange(60,200),intercept+slope*np.arange(60,200), color="r")
plt.xlim([60, 200])
plt.xlabel("1931 crop yield")
plt.ylabel("1932 crop yield")

From the equation we can see that the intercept is equivalent to the crop yield in 1932 (y) in case the yield in 1931 (x) was zero:
$$ y = a0 + b = b$$

In [ ]:
plt.scatter(x=year1_yield, y=year2_yield)
plt.plot(np.arange(0,200),intercept+slope*np.arange(0,200), color="r")
plt.xlim([0, 200])
plt.xlabel("1931 crop yield")
plt.ylabel("1932 crop yield")
print intercept

How is this line fitted? The line is a prediction. It tells you what should be the crop yield in 1932 based on 1931. But it's not perfect. Let's draw how far the predictions are from reality.

In [ ]:
plt.scatter(x=year1_yield, y=year2_yield)
plt.plot(np.arange(60,200),intercept+slope*np.arange(60,200), color="r")
plt.xlim([60, 200])
plt.xlabel("1931 crop yield")
plt.ylabel("1932 crop yield")
plt.vlines(year1_yield, year2_yield,intercept+slope*year1_yield)

We can count the errors

In [ ]:
print np.abs(year2_yield-intercept+slope*year1_yield).sum()

Different lines will have bigger different errors:

In [ ]:
slope2 = slope + 0.15
intercept2 = intercept - 10

plt.scatter(x=year1_yield, y=year2_yield)
plt.plot(np.arange(60,200),intercept2+slope2*np.arange(60,200), color="r")
plt.xlim([60, 200])
plt.xlabel("1931 crop yield")
plt.ylabel("1932 crop yield")
plt.vlines(year1_yield, year2_yield,intercept2+slope2*year1_yield)
print np.abs(year2_yield-intercept2+slope2*year1_yield).sum()

The fitting procedure is minimizing those errors trying to find a line that best describes the data.

In practice sum of absolute values of errors is not minimized - it's the sum of squares

In [ ]:
print ((year2_yield-intercept2+slope2*year1_yield)**2).sum()

In [ ]:
((year2_yield-intercept2+slope2*year1_yield)**2).sum()

In practice this means that big errors will make much bigger impact on the fit than small errors.

Why square not absolute value? This comes from the fact that linear regression model is using Gaussian random noise.

$$ y = ax + b + \epsilon$$
where $\epsilon = N(0,1)$ (is a Gaussina with mean zero and variance 1)

Note that fitting a linear model has a close solution. This means that we can calculate optimal slope and intercept parameters without having to try multiple different lines.

To assess how well we can predict values of one variable with the values of another we can use the correlation value

In [ ]:
r_value

Correlation is nothing else than covariance of the two variables normalized by their variance. It varies from -1 (anticorrealted), through 0 (uncorrelated), to 1 (fully correlated)

$$P_{ij} = \frac{ C_{ij} } { \sqrt{ C_{ii} * C_{jj} } }$$

In [ ]:
np.corrcoef(x=year1_yield, y=year2_yield)

Correlation coefficient has another interesting property - when squared it describes the percentage of variance of variable Y explained by one varaince in variable X (and vice versa) 

In [ ]:
r_value**2*100

There is also a p-value. It is directly related to the r_value and the number of samples.

## Multiple linear regression

In [ ]:
prestige = sm.datasets.get_rdataset("Duncan", "car", cache=True).data

In [ ]:
ds = sm.datasets.get_rdataset("Duncan", "car", cache=True)
print ds.__doc__

In [ ]:
prestige.head()

How well can we describe predict prestige using education and icome?

In [ ]:
prestige_model = ols("prestige ~ income + education", data=prestige).fit()
print(prestige_model.summary())

Multiple linear regression (or General Linear Models - GLM) are based on the premise of a matrix of variables describing the data:
$$ Y = \beta X + \epsilon $$
where $\beta$ are the coefficient and X is the design matrix. In our case the design matrix looks this way

In [ ]:
plt.matshow(prestige_model.model.exog)
plt.xticks(range(3), prestige_model.model.exog_names, rotation=-25)
plt.colorbar()

Note that on top of our two regressors of interest ("income" and "education") we also have "intercept" just like in regural linear regrossion with one variable.

In fact we can recreate the same results from our previous example using GLM.

In [ ]:
crop_model = ols("Y2 ~ Y1", data=immer_ds.data).fit()
print(crop_model.summary())

In [ ]:
plt.matshow(crop_model.model.exog)
plt.xticks(range(2), crop_model.model.exog_names, rotation=-25)
plt.colorbar()

You can also express two sample T test in GLM

In [ ]:
immer_subset = immer_data[np.logical_or(immer_data.Var == "M", immer_data.Var == "T")]

In [ ]:
subset_model = ols("Y1 ~ Var", data=immer_subset).fit()
print(subset_model.summary())

In [ ]:
plt.matshow(subset_model.model.exog)
plt.xticks(range(2), subset_model.model.exog_names, rotation=-25)
plt.colorbar()

This model can be also used for more than two "groups"

In [ ]:
full_model = ols("Y1 ~ Var", data=immer_ds.data).fit()
print(full_model.summary())

In [ ]:
plt.matshow(full_model.model.exog)
plt.xticks(range(len(full_model.model.exog_names)), full_model.model.exog_names, rotation=-25)
plt.colorbar()

We can now explicitly test hypotheses such as is there a difference between manchuria and trebi.

In [ ]:
full_model.f_test("Var[T.P] = Var[T.T]")

Which is equivalent to

In [ ]:
full_model.f_test([1,0,0,0,0])

There are also other ways of coding categorical variables in design matrices

## Shared variance and colinearity

In [ ]:
data = sm.datasets.longley.load_pandas().data
data.head()

In [ ]:
print sm.datasets.longley.NOTE

First we fit a model with one variable

In [ ]:
model_1 = ols("TOTEMP ~ POP", data=data).fit()
print(model_1.summary())

Lots of variance explained and high t value!

Lets try another variable

In [ ]:
model_2 = ols("TOTEMP ~ GNP", data=data).fit()
print(model_2.summary())

Even more variance explained and even higher t value!

Let's put both variables in the model

In [ ]:
model_1_plus_2 = ols("TOTEMP ~ POP + GNP", data=data).fit()
print(model_1_plus_2.summary())

The t values decreased drastically! Effect of the population (POP) changed direction!

In [ ]:
sns.jointplot("GNP", "POP", data=data)

It's because the two variables are highly correlated. t values and coefficients reported by a GLM model correspond only to the unique variance. Varaince shared by the independen variables are not assigned to any of them. Note however overall model performance ($R^2$) increased.

In [ ]:
sns.jointplot("TOTEMP", "POP", data=data)

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(data["GNP"], data["POP"])

In [ ]:
predicted = intercept+slope*data["GNP"]
data["POPwoGNP"] = data["POP"] - predicted

In [ ]:
sns.jointplot("TOTEMP", "POPwoGNP", data=data)